In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
sns.set_palette("colorblind")

In [26]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential   #, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Rescaling, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from tensorflow.keras.initializers import GlorotUniform

In [154]:
def get_filepath(article_id):
    """
    Returns the filepath of the image for the given article_id, e.g. 'samples/10/108775015.jpg'
    article_id: (str) article_id e.g. '108775015'
    """
    filepath = 'samples/' + article_id[:2] + '/' + '0' + article_id + '.jpg'
    return filepath

In [211]:
def list_files(dir):
    """
    Returns list of filepaths in the given directory (dir) excluding hidden files
    """
    file_list = []
    for root, dirs, files in os.walk(dir):
        for file in files:
            if not file.startswith('.'):
                file_list.append(os.path.join(root, file))
    return file_list
len(list_files('samples'))

5

In [198]:
# !kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f "images"

404 - Not Found


# Personalized Fashion Recommendations
## Baseline model for image recognition

The images are RGB color coded.

### Load the list of the samples

In [155]:
with open("filename_product_group_name.txt", "r") as file:
    lines = file.readlines()[1:]
    data = [line.strip().split(",") for line in lines]
    df = pd.DataFrame(data, columns=["article_id", "product_group_name"])

In [156]:
df['article_filepath'] = df['article_id'].apply(get_filepath)

### Preprocess

In [166]:
# The product_group_name to have
categories_to_have = \
['Garment Upper body', 'Garment Lower body', 'Garment Full body', 'Accessories', 'Underwear',\
 'Shoes', 'Swimwear', 'Socks & Tights', 'Nightwear']

le = LabelEncoder()
le.fit(categories_to_have)
labels = df['product_group_name']
labels = pd.Series(data=le.transform(labels), index=labels.index)

In [158]:
X = df['article_filepath']

### Load and preprocess the images

In [159]:
# List of image paths
#image_paths = ["samples/sample_tshirt.jpg", "samples/sample_tshirt.jpg"]
image_paths = X[:3].values
# List to store the loaded images
images = []

In [141]:
TARGET_SIZE = (100, 100)

In [161]:
# Load the images and convert them to numpy arrays
for path in image_paths:
    image = load_img(path=path, target_size=TARGET_SIZE)
    image = img_to_array(image)
    images.append(image)

# Convert the list of images to a numpy array
images = np.stack(images, axis=0)

In [163]:
images.shape

(3, 100, 100, 3)

In [59]:
# # Fake labels
# labels = np.random.choice(range(NUMBER_OF_CLASSES), size=len(images), replace=True)
# labels.shape

(2,)

### Train-test split

In [173]:
#images_train, images_test, labels_train, labels_test = train_test_split(images, labels, train_size=2)

images_train = images
labels_train = labels

### Model building

In [171]:
NUMBER_OF_CLASSES = len(np.unique(labels))
NUMBER_OF_CLASSES

9

In [172]:
# Define the model
baseline_model = Sequential(layers=[
    # Rescale the image in the [0, 255] range to be in the [0, 1] range
    Rescaling(scale=1./255, input_shape=images.shape[1:]),
    
    # The first convolutional layer
    Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    
    # The max pooling layer
    MaxPooling2D(pool_size=(2, 2)),
    
    # Flatten the output
    Flatten(),
    
    # Fully connected layer
    Dense(units=32, activation='relu'),
    
    # The output layer with units=NUMBER_OF_CLASSES. Sum of outputs equals to 1.
    Dense(NUMBER_OF_CLASSES, activation='softmax')
], name="baseline_model")

In [174]:
# Check if the input and output of the model are correct
assert baseline_model.input_shape[1:] == images.shape[1:]
assert baseline_model.output_shape[1:] == (NUMBER_OF_CLASSES,)

In [175]:
# Compile the model
baseline_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [176]:
# Make Early Stopping and Checkpoints
model_checkpoint = ModelCheckpoint(filepath='best_model_baseline_CNN.h5',
                                   monitor='val_loss',
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='min',
                                   verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=5, mode='min')

### Model fitting

In [177]:
baseline_model.fit(x=images_train, y=labels_train, batch_size=1, epochs=100, validation_split=0.1,
          callbacks=[early_stop, model_checkpoint])

Epoch 1/100
1/2 [==============>...............] - ETA: 0s - loss: 2.4853 - accuracy: 0.0000e+00
Epoch 1: val_loss improved from inf to 0.00000, saving model to best_model_baseline_CNN.h5
2/2 [==============================] - 1s 292ms/step - loss: 1.2426 - accuracy: 0.5000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/100
1/2 [==============>...............] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2: val_loss did not improve from 0.00000
2/2 [==============================] - 0s 37ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/100
1/2 [==============>...............] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3: val_loss did not improve from 0.00000
2/2 [==============================] - 0s 34ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/100
1/2 [==============>...............] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4: val_loss did 

In [214]:
# !git branch
# !git add .
# !git commit -m "baseline_model_CNN_kuzma_colab.ipynb is most correct"
# !git push origin kuzma

[kuzma 552bd95] baseline_model_CNN_kuzma_colab.ipynb is most correct
 2 files changed, 913 insertions(+), 12 deletions(-)
 create mode 100644 baseline_model_CNN_kuzma_colab.ipynb
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 6.82 KiB | 6.82 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/kuzmatsukanov/kuzma_omri_noa_data_project.git
   2074ebe..552bd95  kuzma -> kuzma
